# トレンドレコメンド（仮）

## 概要

- 世間で話題になっているトレンドを抽出し、トレンドに関連した商品を自動抽出する
- 使用したAPI （TwitterAPI, NewsAPI, 楽天API）

## 処理の流れ

1. 以下のいずれかの方法により、トレンドワードを取得する
- GoogleトレンドのWebページをクローリング・スクレイピングし、Google検索で急上昇中のワードを取得する（https://trends.google.co.jp/trends/trendingsearches/realtime?geo=JP&category=all）
- TwitterAPIを使用し、Twitterでトレンドになっているワードを取得する
- NewsAPIを使用し、国内ニュースでトレンドになっているニュースを取得する（これはワードではなく、ニュース記事を取得）
<br>
※ 各処理で必要となる処理は関数化済みだが、今回はGoogleトレンドのパターンで実験する


2. トレンドワードに関連するニュース記事をNewsAPIを使用して取得する
- 1で取得したトレンドワードでニュース記事を検索する
- ニュース記事をタイトルと概要を取得する（本文取得はAPIではできない）

3. トレンドワードに関連するツイートをTwitterAPIを使用して取得する
- 取得したトレンドワードでツイートを検索する

4. トレンドワード・ニュース記事（複数）・ツイート（10件）を一つのテキストにまとめる
5. 楽天APIを使用し、楽天ブックスで販売する書籍のタイトル・著者・概要を取得する
- 一度に楽天APIで取得できる商品数には制限がある為、事前にAPIを一日叩き続けて取得できた10万件近い書籍のデータを使用する
- 書籍データはpandasのデータとして保持し、pickleファイル化しておく（実際はDBに保存しておきたい）

6. 5で取得した書籍情報（タイトル・著者・概要）を一つのテキストにまとめる

7. 4と6をMecabで形態素解析し、名詞だけを取得する

8. TFIDFのベクトル化して、トレンドに関連するテキスト（トレンドワード・ニュース記事・ツイート）に一番類似する商品をCOS類似度を算出し求める

## 以下、処理内容

In [1]:
# 必要なライブラリをインストール（AWSで環境をゼロから作り直す時必要）
# !pip install newsapi-python
# !pip install twitter
# !pip install selenium
# !pip install feedparser

### 前処理

In [132]:
# 各APIで必要なライブラリをインポート
from newsapi import NewsApiClient
from twitter import *
import requests

In [79]:
# その他、必要なライブラリをインポート
import datetime
import re
from selenium import webdriver
import time
import feedparser
import random
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [80]:
# API_KEYの設定ファイルを読み込み
# ※ APIを使用する為に必要となるキーとパスをまとめた設定ファイルを別途作成し、それを呼び出している
import configparser
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [81]:
# postgresqlに接続する為のライブラリ
import psycopg2
from sqlalchemy import create_engine

### news_api

In [82]:
# news_apiから、検索したキーワードに関連するニュース情報を返す関数
def keyword_news_search(\
                         api_key = config['NEWS_API']['KEY'],\
                         search_word = '', \
                         start_date = datetime.datetime.now().strftime('%Y-%m-%d'),\
                         end_date = datetime.datetime.now().strftime('%Y-%m-%d'), \
                         sort_type = 'popularity'\
                        ):
    
    # news_apiを初期化
    news_api_search = NewsApiClient(api_key=config['NEWS_API']['KEY'])
    
    # search_wordで指定したキーワードに関するニュースを検索する
    result_search_news = news_api_search.get_everything(qintitle=search_word, from_param=start_date ,to=end_date, sort_by=sort_type)
    
    return result_search_news

In [83]:
# news_apiから、トップニュース情報を返す関数
def headlines_news(\
                         api_key = config['NEWS_API']['KEY'],\
                         target_category = 'entertainment', \
                         target_country = 'jp'
                        ):
    
    # news_apiを初期化
    news_api_headlines = NewsApiClient(api_key=config['NEWS_API']['KEY'])
    
    # 指定したオプションに従ったトップニュースを検索する     
    # Possible options: business entertainment general health science sports technology . 
    # Note: you can't mix this param with the sources param.
    result_headlines_news = news_api_headlines.get_top_headlines(category=target_category, country=target_country)
    
    return result_headlines_news

### twitter_api

In [84]:
# twitter_apiから、ツイートトレンドの単語を返す関数
def twitter_trends_search(\
                          CK = config['TWITTER_API']['CONSUMER_KEY'],\
                          CS = config['TWITTER_API']['CONSUMER_SECRET'],\
                          AT = config['TWITTER_API']['ACCESS_TOKEN'],\
                          AS = config['TWITTER_API']['ACCESS_TOKEN_SECRET'],\
                          area_id = 23424856\
                         ):
    
     # twitter_apiを初期化
    twitter_api_trends = Twitter(auth = OAuth(AT,AS,CK,CS))
        
     # 指定したエリアのツイッタートレンドを検索する
    twitter_trends_results = twitter_api_trends.trends.place(_id = area_id)
                    
    return twitter_trends_results

In [85]:
# twitter_apiから、検索したキーワードに関連するツイートを返す関数
def twitter_search(\
                          CK = config['TWITTER_API']['CONSUMER_KEY'],\
                          CS = config['TWITTER_API']['CONSUMER_SECRET'],\
                          AT = config['TWITTER_API']['ACCESS_TOKEN'],\
                          AS = config['TWITTER_API']['ACCESS_TOKEN_SECRET'],\
                          search_keyword = 'twitter',\
                          search_lang = 'ja',\
                          search_type = 'mixed',\
                          search_count = 100\
                         ):
    
     # twitter_apiを初期化
    twitter_api_search = Twitter(auth = OAuth(AT,AS,CK,CS))
        
     # 指定したエリアのツイッタートレンドを検索する
    search_results = twitter_api_search.search.tweets(q=search_keyword, lang=search_lang, result_type=search_type, count=search_count)
                    
    return search_results

### Googleトレンド（クローリング・スクレイピング）

In [234]:
def google_trends_search(\
                        base_url = 'https://trends.google.co.jp/trends/trendingsearches/realtime',\
                        search_geo = 'JP',\
                        search_category = 'all',
                        ):
    
    try:

        # 検索するURLを作成する
        url = '{i}?geo={j}&category={k}'.format(i=base_url, j=search_geo, k=search_category)

        # ブラウザを開く
        driver = webdriver.Chrome()

        # Googleトレンドの検索TOP画面を開く （表示サイズも指定しておく）
        driver.get(url)
        driver.set_window_size(2500,1000)

        # 2〜4秒間、ページが開かれるのを待つ
        time.sleep(random.randint(2,4))

        # TOP画面より、必要な要素のデータを取得する
        trends_keywords = driver.find_elements_by_class_name('title')
        summary_text = driver.find_elements_by_class_name('summary-text')
        source_text = driver.find_elements_by_class_name('source-and-time')

        # 全ての要素が取得できた場合、処理を進める
        while bool(len(trends_keywords) == 0 or  len(summary_text) == 0 or len(source_text) == 0):
            time.sleep(10)
            trends_keywords = driver.find_elements_by_class_name('title')
            summary_text = driver.find_elements_by_class_name('summary-text')
            source_text = driver.find_elements_by_class_name('source-and-time')
        else:
            trends_keyword = []
            trends_abst = []
            inyo_url = []
            inyo_site = []
            for i in range(len(trends_keywords)):
                trends_keyword.append(trends_keywords[i].text)
                trends_abst.append(summary_text[i].text)
                inyo_url.append(summary_text[i].find_element_by_tag_name("a").get_attribute("href"))
                inyo_site.append(source_text[i].text.split()[0])

        # 各トレンドごとに詳細情報を取得する為、要素をクリックし必要なHTMLを表示させる（その為のクリックする要素を取得している）
        trend_click = driver.find_elements_by_class_name('feed-item')

        # 蓄積用のリストを用意しておく
        google_trend_news_list = []
        google_trend_words_list = []

        # トレンドごとに詳細情報を取得する為、要素をクリックし必要なHTMLを表示させる
        for trend in trend_click:
            trend.click()

            # 1秒間、ページが開かれるのを待つ
            time.sleep(3)
            
            # 引用ニュースを取得
            google_trend_news_tmp = driver.find_elements_by_class_name('item-title')
            google_trend_news = ''
            for news in google_trend_news_tmp:
                google_trend_news = google_trend_news + news.text

            # 関連ワードを取得
            google_trend_words_tmp = driver.find_elements_by_class_name('list')
            google_trend_words = google_trend_words_tmp[0].text.replace(" ","")

            # リストに追加
            google_trend_news_list.append(google_trend_news)
            google_trend_words_list.append(google_trend_words) 

        driver.quit()
    
    except:
        pass
    
    google_trend_data = {'goole_keyword': trends_keyword, 'google_abst': trends_abst, 'google_url': inyo_url, 'google_site': inyo_site, \
                        'google_news':google_trend_news_list, 'google_words':google_trend_words_list}
    
    return google_trend_data

In [235]:
google_trends = google_trends_search()

## 必要な処理は用意できたので、ここからレコメンドを作っていく。

#### ちょっとしたデータ加工

In [236]:
google_trend_word = []
for i in google_trends['goole_keyword']:
    google_trend_word.append(i.split(' • '))

In [237]:
google_trend_news = google_trends['google_news']
google_trend_words = google_trends['google_words']

#### 【NesAPIを使用】トレンドワードに基づいて、関連ニュースを取得する

In [238]:
sample_news = []
for j in google_trend_word:
    sample_news.append(keyword_news_search(search_word = j[0]))
    time.sleep(1)

In [239]:
news_text = []
for k in sample_news:
    news_text_tmp = ''
    for o in k['articles']:
        news_text_tmp += (o['title'] +","+ str(o['description']))
    news_text.append(news_text_tmp)

#### 【twitterAPIを使用】トレンドワードに基づいて、関連ツイートを取得する

In [240]:
twtter_text = []
for p in google_trend_word:
    twtter_text.append(twitter_search(search_keyword=p[0], search_count = 10))
    time.sleep(1)

In [241]:
twitter_text_list = []
for q in twtter_text:
    
    twitter_text_tmp = ''
    for i in range(len(q["statuses"])):
        twitter_text_tmp += q["statuses"][i]["text"]
        
    twitter_text_list.append(twitter_text_tmp)

#### googleトレンドのページに記載されているテキストは重みをつける（✖️3）

In [242]:
#　ちょっとデータ加工
google_trend_word_text = []
for r in google_trend_word:
    google_trend_word_text_tmp = ','.join(r)
    google_trend_word_text.append(google_trend_word_text_tmp)

In [243]:
test_text = []
for s in range(len(google_trend_word_text)):
#     test_text.append((google_trend_word_text[s] + google_trend_news[s] + google_trend_words[s]) * 3 + news_text[s] + twitter_text_list[s])
    test_text.append((google_trend_word_text[s] + google_trend_news[s] + google_trend_words[s]) * 3 + news_text[s])

## 楽天APIで取得済みの商品リストを持ってくる

#### 楽天APIで書籍情報を取得し続ける処理は別途作成し実行ずみ。.pickleのファイルにしているので、ここで読み込み使用する。

In [244]:
# PostgreSQL Server へ接続（必要情報は削除）
conn = psycopg2.connect('host=localhost port=5432 dbname=trend_books user=postgres password=panchi1')

In [245]:
# データベースの接続情報（必要情報は削除）
connection_config = {
    'user': 'postgres',
    'password': 'panchi1',
    'host': 'localhost',
    'port': '5432', # なくてもOK
    'database': 'trend_books'
}
engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**connection_config))

In [246]:
# PostgreSQLに楽天APIから集計したデータを書き込む

# import pickle
# with open('df_all_duplicated.pickle','rb') as f:
#     book_list = pickle.load(f)
# book_list_db.to_sql('book', con=engine, if_exists='replace', index=False)

In [247]:
# PostgreSQLにトレンド情報のデータを書き込む
df_trend = pd.DataFrame({'time': pd.to_datetime(datetime.datetime.now(), format='%y%m%d %H:%M'),\
                         'rank': [i+1 for i in range(len(google_trend_news))],\
                         'google_trend_words': google_trend_word_text,\
                         'google_trend_news': google_trend_news,\
                         'google_trend_words_related': google_trend_words,\
                         'news': news_text,\
                         'tweet': twitter_text_list})

df_trend.to_sql('trend', con=engine, if_exists='append', index=False)

In [248]:
# PostgreSQLに接続する
connection = psycopg2.connect(**connection_config)

In [249]:
# テーブルの全レコードを格納する
book_list = pd.read_sql(sql="SELECT * FROM book;", con=conn)

In [250]:
test_books = (book_list['title']+","+book_list['author']+",")*5+","+book_list['subTitle']+","+book_list['itemCaption']

In [251]:
df1 = pd.DataFrame()
df1['test'] = test_books

In [252]:
df2 = pd.DataFrame(test_text, columns=['test'])

In [253]:
# df3 = df1
df3 = df1.append(df2).reset_index()

#### 自然言語処理で、テキストをベクトルに変換していく

In [254]:
from sklearn import preprocessing

In [255]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [256]:
import MeCab
import re

In [257]:
# import janome
# from janome.tokenizer import Tokenizer

# tokenizer = Tokenizer()

# for token in tokenizer.tokenize("大村競艇"):
#     print("    " + str(token))

In [258]:
# MeCabの辞書にnelogdを指定する
m = MeCab.Tagger(r"-r C:\Users\kazum\Desktop\GS\trend_recommend\database_test\api\mecabrc-u")

In [259]:
# あまり関係のないと思われる数字を全て0に置き換える関数
def replace_number_to_zero(text):
    changed_text = re.sub(r'[0-9]+', "0", text) #半角
    changed_text = re.sub(r'[０-９]+', "0", changed_text) #全角
    return changed_text

# 数字を0に置換
df3['review_number_to_zero'] = df3['test'].map(replace_number_to_zero)

In [260]:
# 分かち書きした結果を返す関数
def leaving_space_between_words_column(text):
    
    nouns = []
        
    for line in m.parse(text).splitlines():
        if len(line.split()) > 0:
            if "名詞" in line.split()[-1]:
                nouns.append(line)
                
    meishi_list = []
    for str in nouns:
        try:
#             meishi_list.append(str.split()[2])
            meishi_list.append(str.split(",")[6])
        except:
            meishi_list.append("")
    
#     return ','.join(meishi_list)
    return ' '.join(meishi_list)

# 分かち書きしたカラムをdfに追加する
df3['lsbw'] = df3['review_number_to_zero'].map(leaving_space_between_words_column)

In [261]:
# 50万冊の書籍情報をベクトル化
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df3['lsbw'])

In [262]:
# トレンドのテキスト情報もベクトル化
test_item = vectorizer.transform(df2["test"])

In [263]:
# 特徴語を取得する
values = test_item.toarray()
feature_names = vectorizer.get_feature_names()
dfsss = pd.DataFrame(values, columns = feature_names)
df_0 = dfsss.T

feature_list = []

for i in df_0.columns:
    feature_list.append(df_0[df_0[i]>0][i].sort_values(ascending=False)[:3])

In [287]:
# トレンドに関連した商品を抽出
for i,u in enumerate(test_item):
    similarity = cosine_similarity(u, X[:-20])
    similarity_list = similarity[0].tolist()
   
    print("<<トレンド情報>>")
    print(google_trends["google_abst"][i])
    print('-----')   
    print("<<関連書籍>>")
    
    tmp_max_value = 0
    
    count = 0

    for v in range(-1,-4,-1):
        max_value = sorted(similarity_list)[v]
        index_nums = [n for n, v in enumerate(similarity_list) if v == max_value]
        
        for index_num in index_nums:
            print(count+1)
            print(df3.loc[index_num, "test"])
            
            count = count + 1
            
            if count == 3:
                break
        
        if count == 3:
            count = 0
            break
            

    print('-----')
    print('-----')

<<トレンド情報>>
米大リーグ歴代２位の７５５本塁打、ハンク・アーロンさん死去…８６歳
-----
<<関連書籍>>
1
【輸入楽譜】ベスト・オブ・ハンク・ウィリアムズ - 第2版,,【輸入楽譜】ベスト・オブ・ハンク・ウィリアムズ - 第2版,,【輸入楽譜】ベスト・オブ・ハンク・ウィリアムズ - 第2版,,【輸入楽譜】ベスト・オブ・ハンク・ウィリアムズ - 第2版,,【輸入楽譜】ベスト・オブ・ハンク・ウィリアムズ - 第2版,,,,
2
ウルフ・ウォーズ,ハンク・フィッシャー/朝倉裕,ウルフ・ウォーズ,ハンク・フィッシャー/朝倉裕,ウルフ・ウォーズ,ハンク・フィッシャー/朝倉裕,ウルフ・ウォーズ,ハンク・フィッシャー/朝倉裕,ウルフ・ウォーズ,ハンク・フィッシャー/朝倉裕,,オオカミはこうしてイエローストーンに復活した,市民、政治家、畜産業界、自然保護団体の思惑が複雑に絡み合う困難な道をどう切り開いてゴールを目指すか。オオカミ再導入実現の立役者が苦闘と創意工夫の２０年をつぶさに物語る。
3
【輸入楽譜】ウィリアムス, Hank: ハンク・ウィリアムス: Complete,ウィリアムス, Hank,【輸入楽譜】ウィリアムス, Hank: ハンク・ウィリアムス: Complete,ウィリアムス, Hank,【輸入楽譜】ウィリアムス, Hank: ハンク・ウィリアムス: Complete,ウィリアムス, Hank,【輸入楽譜】ウィリアムス, Hank: ハンク・ウィリアムス: Complete,ウィリアムス, Hank,【輸入楽譜】ウィリアムス, Hank: ハンク・ウィリアムス: Complete,ウィリアムス, Hank,,,
-----
-----
<<トレンド情報>>
坂上忍 昨年結婚、野呂佳代の「僕が婚姻届の証人です」夫の正体も明かし松本「知らんかった」
-----
<<関連書籍>>
1
別冊月刊真木よう子Noise,藤代 冥砂,別冊月刊真木よう子Noise,藤代 冥砂,別冊月刊真木よう子Noise,藤代 冥砂,別冊月刊真木よう子Noise,藤代 冥砂,別冊月刊真木よう子Noise,藤代 冥砂,,,
2
桐谷健太 2nd PHOTO BOOK 『 CHELSEA 』,関根虎洸/桐谷健太,桐谷健太 2nd PHOTO BOOK 『 CHELSEA 』,関根虎洸/

<<トレンド情報>>
東京事変の2021年新曲第1弾“闇なる白”が明日配信リリース！吉澤智子書き下ろしNHKドラマ『ドリームチーム』主題歌
-----
<<関連書籍>>
1
僕らだけの主題歌,,僕らだけの主題歌,,僕らだけの主題歌,,僕らだけの主題歌,,僕らだけの主題歌,,,,
2
LTBS173　FOUL／東京事変,,LTBS173　FOUL／東京事変,,LTBS173　FOUL／東京事変,,LTBS173　FOUL／東京事変,,LTBS173　FOUL／東京事変,,,,
3
邦画の主題歌による合唱はつらいよ,,邦画の主題歌による合唱はつらいよ,,邦画の主題歌による合唱はつらいよ,,邦画の主題歌による合唱はつらいよ,,邦画の主題歌による合唱はつらいよ,,,,
-----
-----
<<トレンド情報>>
ＵＦＣマクレガー復帰戦へ「ポワリエを２度倒す」
-----
<<関連書籍>>
1
マクレガーとホークの世代,石原郁子,マクレガーとホークの世代,石原郁子,マクレガーとホークの世代,石原郁子,マクレガーとホークの世代,石原郁子,マクレガーとホークの世代,石原郁子,,ユアン・マクレガー　スティーヴン・ドーフ　イーサン,
2
青木真也の柔道＆柔術入門,青木真也,青木真也の柔道＆柔術入門,青木真也,青木真也の柔道＆柔術入門,青木真也,青木真也の柔道＆柔術入門,青木真也,青木真也の柔道＆柔術入門,青木真也,,The　Finish！！,本書は青木真也が考える『組み技格闘技』を『道衣』を通して表現したものである。柔道、柔術、サンボ、サブミッションレスリング…すべての格闘技愛好家に贈る。
3
戦略の形成　下,ウィリアムソン・マーレー/マクレガー・ノックス,戦略の形成　下,ウィリアムソン・マーレー/マクレガー・ノックス,戦略の形成　下,ウィリアムソン・マーレー/マクレガー・ノックス,戦略の形成　下,ウィリアムソン・マーレー/マクレガー・ノックス,戦略の形成　下,ウィリアムソン・マーレー/マクレガー・ノックス,,支配者、国家、戦争,戦略の策定を論じる際、しばしばクラウゼヴィッツやリデルハート等、戦略思想家の影響が語られてきたが、本書はこれに疑問を呈する。戦略とは、敵・味方の相互作用であり、不可測な要素が支配する領域であるので、明確で論理的な原理や原則は存在し得ないと指摘する。地理や歴

In [290]:
for i in range(len(feature_list)):
    print("トレンド情報：　", google_trends["google_abst"][i])
    print("特徴語：　",feature_list[i].index)
    print("---")

トレンド情報：　 米大リーグ歴代２位の７５５本塁打、ハンク・アーロンさん死去…８６歳
特徴語：　 Index(['ハンク', 'アトランタ', 'アルビレックスbc'], dtype='object')
---
トレンド情報：　 坂上忍 昨年結婚、野呂佳代の「僕が婚姻届の証人です」夫の正体も明かし松本「知らんかった」
特徴語：　 Index(['野呂佳代', '真木よう子', 'ダウンタウンなう'], dtype='object')
---
トレンド情報：　 岩田剛典 “I Don't Like Mondays.” YU、幼馴染対談実現に「記念すべき日」
特徴語：　 Index(['名も無き世界のエンドロール', 'don', 'like'], dtype='object')
---
トレンド情報：　 miHoYoの『崩壊3rd』がApp Store売上ランキングで327位→19位に急上昇 コラボ戦乙女にアスカ登場の「エヴァンゲリオン」コラボ開催で
特徴語：　 Index(['エヴァンゲリオン', 'アスカ', 'ラングレー'], dtype='object')
---
トレンド情報：　 元欅坂46今泉佑唯ファンがワタナベマホトとの結婚を祝福できない理由
特徴語：　 Index(['ワタナベマホト', '今泉佑唯', 'https'], dtype='object')
---
トレンド情報：　 アイナ・ジ・エンド、「THE FIRST TAKE」第86回にて“オーケストラ”を披露
特徴語：　 Index(['エンド', 'first', 'take'], dtype='object')
---
トレンド情報：　 応援購入サービス「Makuake」、「オンライン催事」機能の提供を開始！ ～第一弾は大丸東京店セレクトの銘菓子店を集めた「オンライン催事・大丸東京店 ...
特徴語：　 Index(['makuake', 'https', 'jp'], dtype='object')
---
トレンド情報：　 “コンプライアンス”違反を指摘する視聴者からの手紙が引き金となり、過剰な役柄の滝藤賢一は降板させられる事態に...：バイプレイヤーズ
特徴語：　 Index(['観月ありさ', 'バイプレイヤーズ', '大倉孝二'], dtype='object')
---
トレンド

In [266]:
# 書籍データを蓄積するリストを用意
search_list = {}

url = "https://app.rakuten.co.jp/services/api/BooksTotal/Search/20170404"

# APIでデータ集計を行うページ番号について、初期値を設定
page_num = 1

for i in range(len(feature_list)):
    for k,feature_name in enumerate(feature_list[i].index):

        # URLに付与するパラメータ
        payload = {
            'applicationId': config["RAKUTEN_API"]["id"],
            'hits': 5,
            'page': page_num,
            'formatVersion': 2,
            'keyword': feature_name
        }

        r = requests.get(url, params=payload) 
        resp = r.json()
        for j in range(len(resp["Items"])):
            resp["Items"][j]["rank"] = i + 1
            search_list[feature_name] = resp['Items']

        time.sleep(1)


In [289]:
tmp = 1
print("<<各トレンドの文章に含まれる上位の特徴語と、特徴語で楽天APIの検索をかけた結果>>")

print("-----")
for k,v in search_list.items():
    if v[0]["rank"] != tmp:
        print("-----") 
    print(k,":",v[0]["title"])
    tmp = v[0]["rank"]

<<各トレンドの文章に含まれる上位の特徴語と、特徴語で楽天APIの検索をかけた結果>>
-----
ハンク : デュオ
アトランタ : アトランタ・ノースカロライナ・サウスカロライナ・テネシー・アラバマ・テキサス便利（VOL．16）
-----
野呂佳代 : ハッピーメール
真木よう子 : 別冊月刊真木よう子Noise
-----
名も無き世界のエンドロール : 名も無き世界のエンドロール
don : 【楽天ブックス限定先着特典】AGE OF ZOC/DON'T TRUST TEENAGER (CD＋DVD＋スマプラ)(特典内容未定)
like : 春雷 feat. 露崎春女
-----
エヴァンゲリオン : 新世紀エヴァンゲリオン原画集ダイジェスト
アスカ : 百希夜行 (完全生産限定盤 CD＋Blu-ray＋Photobook)
ラングレー : Figuarts mini 式波・アスカ・ラングレー
-----
ワタナベマホト : にゃにゃにゃの本〜YouTuber・ワタナベマホトと猫たち〜
今泉佑唯 : 酔うと化け物になる父がつらい
-----
https : ホームページ改善の解決メソッド37
-----
エンド : エンドロールバック（3）（完）
first : PSYCHO-PASS サイコパス3 FIRST INSPECTOR【Blu-ray】
take : GIVE　＆　TAKE
-----
makuake : Deff WIZ Wireless Charging Tray Qi 最大15W キャメルブラウン
jp : コンフィデンスマンJP プリンセス編 超豪華版【Blu-ray】
-----
観月ありさ : VINGT-CINQ ANS (3CD)
バイプレイヤーズ : SODA (ソーダ) 2021年3月号(表紙:松村北斗)
大倉孝二 : 見えない目撃者
-----
ibm : THE DX
intel : ＜BXNUC10I7FNH ＞第10世代Corei7-10710U（1.1-4.7GHz/6 Core/Intel UHD Graphics）搭載NUCキット、 M.2スロット and 2.5 Drive
in : 【楽天ブックス限定先着特典+先着特典】POP IN CITY ～for covers only～ (初回限定盤 CD＋Blu-r